In [1]:
import pandas as pd

In [2]:
import sys
sys.path.insert(0,"../scoring")

In [3]:
from scoring_bhc import calculate_scores_bhc
from scoring_bhc import compute_overall_score_bhc

In [4]:
from scoring_instructions import calculate_scores_instructions
from scoring_instructions import compute_overall_score_instructions

# Test

In [5]:
# notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/comparison_simple_gpt3.5_2shot_v2.csv")
notes = pd.read_csv("/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/brief_hospital_course/outputs/submission.csv")

In [6]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [7]:
discharge_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge.csv.gz", keep_default_na=False)


In [8]:
notes = notes.merge(discharge_test[['hadm_id',"text"]], on="hadm_id")

In [9]:
notes

,Unnamed: 0,hadm_id,discharge_instruction_gpt,Original,text
0,0,26231944,You were admitted to the hospital for decompen...,"Dear Mr. ___,\n\nIt was a pleasure to care for...",\nName: ___ Unit No: ___\n ...
1,1,28542384,You were admitted to the hospital for a recent...,"Dear Ms. ___, \n We had the pleasure of taking...",\nName: ___ Unit No: ___\n \nA...
2,2,26489329,You were admitted to the hospital for a possib...,You were admitted to the hospital after a with...,\nName: ___ Unit No: ___\n ...
3,3,22641254,You were admitted to the hospital for hypoxemi...,"Dear Ms. ___, \n\nIt was a pleasure caring for...",\nName: ___ Unit No: ___\n ...
4,4,29265750,You were admitted to the hospital for right-si...,"Dear Mr. ___, \n\nYou were admitted to ___ wit...",\nName: ___ Unit No: ...
...,...,...,...,...,...
14692,14692,27445071,"Dear Mr. or Ms. [Patient's Last Name], You wer...","Dear Mr. ___,\n\nYou were admitted after getti...",\nName: ___ Unit No: ___\...
14693,14693,26192891,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___,\n\n You were admitted to ___ af...",\nName: ___ Unit No: ___\n ...
14694,14694,26584893,"Dear Ms. ___, You were admitted to the hospita...","Dear Ms. ___,\n\nIt was a pleasure taking ___ ...",\nName: ___ Unit No: _...
14695,14695,25623241,"Dear Mr. ___, You were admitted to the hospita...","Dear Mr. ___, \n\nYou were admitted to ___ \nb...",\nName: ___ Unit No: ___\n ...


In [10]:
# hadm_id,brief_hospital_course,discharge_instructions

In [11]:
reference = notes[['hadm_id', "Original"]].rename({"Original":"brief_hospital_course"}, axis=1)
generated = notes[['hadm_id', "discharge_instruction_gpt"]].rename({"discharge_instruction_gpt":"brief_hospital_course"}, axis=1)

In [13]:
scores = calculate_scores_bhc(generated,
                 reference,
                 [
                     # "bleu", 
                  "rouge", "bertscore", "meteor"])

Beginning scoring...
rougeScorer initialized
bertScorer initialized


[nltk_data] Downloading package wordnet to /home/vs428/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/vs428/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/vs428/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


meteorScorer initialized
Processed 128/14697 samples.
Processed 256/14697 samples.
Processed 384/14697 samples.
Processed 512/14697 samples.
Processed 640/14697 samples.
Processed 768/14697 samples.
Processed 896/14697 samples.
Processed 1024/14697 samples.
Processed 1152/14697 samples.
Processed 1280/14697 samples.
Processed 1408/14697 samples.
Processed 1536/14697 samples.
Processed 1664/14697 samples.
Processed 1792/14697 samples.
Processed 1920/14697 samples.
Processed 2048/14697 samples.
Processed 2176/14697 samples.
Processed 2304/14697 samples.
Processed 2432/14697 samples.
Processed 2560/14697 samples.
Processed 2688/14697 samples.
Processed 2816/14697 samples.
Processed 2944/14697 samples.
Processed 3072/14697 samples.
Processed 3200/14697 samples.
Processed 3328/14697 samples.
Processed 3456/14697 samples.
Processed 3584/14697 samples.
Processed 3712/14697 samples.
Processed 3840/14697 samples.
Processed 3968/14697 samples.
Processed 4096/14697 samples.
Processed 4224/14697 s

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
compute_overall_score_bhc(scores)